# Importing Required Modules

In [23]:
!pip install inline folium
!pip install inline beautifulsoup4
!pip install inline lxml
!pip install inline requests
!pip install inline geocode
!pip install --upgrade tensorflow
import numpy as np
import pandas as pd
import matplotlib as plt
import json
body = client_0c91842efc1a45b2b53a6830d9c94262.get_object(Bucket='datascience-donotdelete-pr-mrcxrdfxydb92m',Key='onsboundariesgen1v2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

import tensorflow
import sklearn.cluster as KMeans
from bs4 import BeautifulSoup
import requests
import csv
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

  Using cached https://files.pythonhosted.org/packages/b1/ab/546879710e53108c1752c38542c5da41ded62c8c7a63add31d8b9303d3c7/geocode-0.1.0.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/home/dsxuser/.tmp/pip-install-4vf79chd/geocode/setup.py", line 2, in <module>
        from __init__ import __version__
      File "/home/dsxuser/.tmp/pip-install-4vf79chd/geocode/__init__.py", line 3, in <module>
        from geocode import *
      File "/home/dsxuser/.tmp/pip-install-4vf79chd/geocode/geocode.py", line 97
        print latlon(input)
                   ^
    SyntaxError: invalid syntax
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /home/dsxuser/.tmp/pip-install-4vf79chd/geocode/
Requirement already up-to-date: tensorflow in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (1.13.1)
Requirement not upgraded

# Data

##### To analyze and predict what location and type would be best for building or investing in a restaurant we are going to require a number of information:
##### 1. The venues and buildings around Ottawa, this will be provided by the FourSquare API
##### 2. Reviews and succeses of these restuarants, and there types. This will be provided by scraping Yelp Information
##### 3. We are going to require key feautures that allow for a restuarant to be successful, this will be provided by 'the balance of small business'
##### 4. User 'u/serenerdy' from reddit made a dataset for Ethnic resturants and the types they are that will also be utliziled where neccesary

In [51]:
# The code was removed by Watson Studio for sharing.

# Ethnic Restaurant Data
#### This dataset has extensice information about the type of Restaurant, Location, What type it is, and notes about the resturant
#### which will aid the FourSqaure API data

In [53]:
df = pd.read_excel(data,'Ethnic Restaurants')
df.head(11)

,Restaurant,Area,Location,Venue type,Ethnic Fare,Price,Recommendations/Notes,Link to site,Phone
0,7e Oeil,Ottawa East,2-411 McArthur,Sit-Down/Delivery,Afro-Caribbean,DIY,NaN,https://www.facebook.com/7oeil.caribean.food/,NaN
1,Aahar the Taste of India,Ottawa East / West,727 Churchill (west)\n1573 Alta Vista (east),Sit-Down,Indian,DIY,NaN,http://www.aahar.ca/,NaN
2,Adam's Sausage Factory,Ottawa East,1437 Michael St.,Deli,Polish Deli,DIY,NaN,http://www.adamssausages.com/,NaN
3,Ahora,Downtown Ottawa,307 Dalhousie,Sit-Down,Tex/Mex,DIY,NaN,http://www.ahora.ca/,NaN
4,Aiyara Thai,Ottawa South,1590 Walkley Road,Sit-Down,Thai,DIY,"Gorgeous décor, Pineapple Rice dish recommended",https://www.aiyarathaicuisine.ca/,NaN
5,Alirang,Downtown Ottawa,134 Nelson st,Sit-Down,Korean,DIY,"Bibimbap, Bulgogi recommended",https://www.yelp.ca/biz/alirang-hamilton,NaN
6,Ariana Kebab,Downtown / Ottawa South,2677 Alta Vista dr. (south)\n 424 Rideau st (d...,Sit-Down,Afghani,DIY,NaN,http://www.arianakababhouse.com/,NaN
7,Asian Stars,Ottawa West,1380 Clyde ave,Sit-Down,Variety of Asian,DIY,Vegan options available,http://asianstarsrestaurant.com/index.html,NaN
8,Baan Thai,Ottawa West,261 Centerpoint dr.,Sit-Down,Thai,DIY,Smaller location- limited seating,https://baanthairestaurantottawa.ca/,NaN
9,Bienvenidos,Ottawa East,1731 St.Laurent Blvd,Market / Grocer,Latin,DIY,"Empanadas, pupusas, tamales recommended",http://www.bienvenidoslatinmarket.ca/,NaN


# Neighborhood Data, there names and the association Area, Population and ID
##### This dataset was gathered from Ottawa Open Data

In [60]:
body = client_0c91842efc1a45b2b53a6830d9c94262.get_object(Bucket='datascience-donotdelete-pr-mrcxrdfxydb92m',Key='onsboundariesgen1v2.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_2 = pd.read_csv(body, encoding='latin-1')
df_2.head(11)

,ONS_ID,Name,Name_FR,Cen2011POP
0,1,Barrhaven,,17250
1,2,Bayshore - Belltown,,8060
2,3,Beacon Hill South - Cardinal Heights,Côte Beacon Sud - Cardinal Heights,7305
3,4,Beaverbrook,,5180
4,5,Beechwood Cemetery,Cimetière Beechwood,451
5,6,Bells Corners East,Bells Corners Est,5110
6,7,Bells Corners West,Bells Corners Ouest,4355
7,8,Billings Bridge - Alta Vista,Pont Billings - Alta Vista,11880
8,9,Blackburn Hamlet,,8260
9,10,Borden Farm - Stewart Farm - Parkwood Hills - ...,Ferme Borden - Ferme Stewart - Côtes Parkwood ...,10135
